**Title:** Datafun-06-Eda   
**Author:** Jacob Sanders  
**Date:** February 2026  

**Purpose:** To determine if there is a correlation between students' chapter  
 test scores and their State Test predictor scores.  I also added students'  
  attendance percentage for the year as an interesting data point.

In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Load your CSV file
df = pd.read_csv('data/math_class_data.csv')

# Inspect first rows of the DataFrame
print(df.head())

print(df.describe())

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 200)         # Expand display width
pd.set_option('display.max_colwidth', None) # Show full content in each column

df.dtypes


   Student_ID Student_Name_First Student_Name_Last_Initial  Chapter_1_Test  Chapter_2_Test  Chapter_3_Test  Chapter_4_Test  Chapter_5_Test  Chapter_6_Test  Chapter_7_Test  Fall_MAP_score  \
0           1                Cat                        LA             100             100              92              93             100              89              95              84   
1           2              Viper                         A              91              70              56              57              96              78              85              75   
2           3            Chicken                        BA              81             100              83              67              34              67              75              53   
3           4               Goat                         C              81              50              64              80              60              33              80              70   
4           5                Hog                  

Student_ID                   int64
Student_Name_First             str
Student_Name_Last_Initial      str
Chapter_1_Test               int64
Chapter_2_Test               int64
Chapter_3_Test               int64
Chapter_4_Test               int64
Chapter_5_Test               int64
Chapter_6_Test               int64
Chapter_7_Test               int64
Fall_MAP_score               int64
Winter_MAP_score             int64
Attendance_Percent_YTD       int64
dtype: object